# =============================================================

# RAG Implementation with MSSQL: Joins

# =============================================================

### Libraries

In [1]:
# !pip install -U langchain langchain_experimental
# !pip install typing-extensions --upgrade

from langchain.llms import LlamaCpp
from langchain_community.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.chains import create_sql_query_chain

### Database Connection in Langchain 

In [2]:
# Connection string
uri = "mssql+pyodbc://@DESKTOP-UM221JG\\SQLEXPRESS/Test?driver=ODBC+Driver+17+for+SQL+Server&Trusted_Connection=yes"
db = SQLDatabase.from_uri(uri, include_tables=["Client_Historical_Orders", "Client_Portfolio"])

In [3]:
# Table information schema
print(db.table_info)


CREATE TABLE [Client_Historical_Orders] (
	order_id INTEGER NOT NULL, 
	client_id INTEGER NULL, 
	client_name VARCHAR(255) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	scrip_name VARCHAR(255) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	order_date DATE NULL, 
	order_price DECIMAL(18, 2) NULL, 
	order_quantity INTEGER NULL, 
	order_type VARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	order_status VARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	CONSTRAINT [PK__Client_H__465962299F72C4D0] PRIMARY KEY (order_id)
)

/*
3 rows from Client_Historical_Orders table:
order_id	client_id	client_name	scrip_name	order_date	order_price	order_quantity	order_type	order_status
1	101	ClientA	RIL	2023-01-10	2800.00	100000	Sell	Open
2	102	ClientB	RIL	2023-01-11	2750.00	50000	Sell	Open
3	103	ClientC	RIL	2023-01-12	2700.00	75000	Sell	Open
*/


CREATE TABLE [Client_Portfolio] (
	client_id INTEGER NULL, 
	client_name VARCHAR(255) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	scrip_name VARCHA

### Loading Mistral-Instruct LLM 

In [4]:
# Defining the model specs for optimized performance 
llama_model_path = r"C:\Users\DELL\Desktop\KSIE\KSIE LLM Chatbot\Models\mistral-7b-instruct-v0.1.Q8_0.gguf"
model_n_ctx = 2048
model_n_batch = 512

# Initializing the LLM object
llm = LlamaCpp(model_path=llama_model_path, n_batch=model_n_batch, n_ctx=model_n_ctx, verbose=True)

AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


### Creating the Retreival Chain

In [5]:
db_chain = SQLDatabaseChain.from_llm(llm, db, use_query_checker=True, verbose=True)

### Execution 

In [6]:
# Example 1: 
# Order: Buy RIL at Rs. 2738
# Match: Open Sell order with original Buy < Rs. 2738
query = "Find all clients with RIL scrip in their portfolio at portfolio buy price lesser than 2738 rupees, who have also placed sell orders with open status for RIL."

db_chain.invoke(query)



> Entering new SQLDatabaseChain chain...
Find all clients with RIL scrip in their portfolio at portfolio buy price lesser than 2738 rupees, who have also placed sell orders with open status for RIL.
SQLQuery:

Llama.generate: prefix-match hit


SELECT Client_Historical_Orders.client_name, Client_Historical_Orders.scrip_name, Client_Portfolio.buy_price
FROM Client_Historical_Orders 
INNER JOIN Client_Portfolio ON Client_Historical_Orders.client_id = Client_Portfolio.client_id 
WHERE Client_Historical_Orders.scrip_name = 'RIL' AND Client_Portfolio.buy_price < CAST(2738 as decimal(18, 2))
AND Client_Historical_Orders.order_type = 'Sell' AND Client_Historical_Orders.order_status = 'Open'
ORDER BY Client_Historical_Orders.client_name ASC;
SQLResult: [('ClientA', 'RIL', Decimal('2700.00'))]
Answer:

Llama.generate: prefix-match hit


ClientA is the only client with RIL scrip in their portfolio at a buy price less than 2738 rupees and who has placed sell orders with an open status for RIL.
> Finished chain.


{'query': 'Find all clients with RIL scrip in their portfolio at portfolio buy price lesser than 2738 rupees, who have also placed sell orders with open status for RIL.',
 'result': 'ClientA is the only client with RIL scrip in their portfolio at a buy price less than 2738 rupees and who has placed sell orders with an open status for RIL.'}